In [2]:
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize
from sklearn.model_selection import train_test_split
from gensim import corpora, models, similarities
from gensim.corpora import Dictionary
from math import sqrt

In [5]:
def make_category_1(file_path):


    f = open(file_path).read()
    data = json.loads(f)
    playlists = data["playlists"]
    df = pd.DataFrame(playlists)
    train, test = train_test_split(df, test_size=0.1)

    r_test_data = {'pid' : list(test['pid']), 'name' : list(test['name']), 'num_holdouts' : list(test['num_tracks']), 'tracks' : [{} for _ in range(test.shape[0])], 'num_samples' : [0 for _ in range(test.shape[0])], 'num_tracks' : test['num_tracks']}
    r_test = pd.DataFrame(r_test_data)
    
    train = train.T.to_dict().values()
    test = test.T.to_dict().values()
    r_test = r_test.T.to_dict().values()
    
    with open('./train.json', 'w+') as f:
        f.write(json.dumps(train))
    with open('./test_R.json', 'w+') as f:
        f.write(json.dumps(test))
    with open('./test.json', 'w+') as f:
        f.write(json.dumps(r_test))
        
        
make_category_1('mpd.slice.0-999.json')

<h1> train_set </h1>

In [6]:
f = open('./train.json').read()
data = json.loads(f)
train_df = pd.DataFrame(data)
train_df

,collaborative,description,duration_ms,modified_at,name,num_albums,num_artists,num_edits,num_followers,num_tracks,pid,tracks
0,false,NaN,11532414,1493424000,Throwbacks,47,37,6,1,52,0,"[{u'pos': 0, u'artist_name': u'Missy Elliott',..."
1,false,NaN,11656470,1506556800,Awesome Playlist,23,21,5,1,39,1,"[{u'pos': 0, u'artist_name': u'Survivor', u'tr..."
2,false,NaN,14039958,1505692800,korean,51,31,18,1,64,2,"[{u'pos': 0, u'artist_name': u'Hoody', u'track..."
3,false,NaN,4335282,1401667200,90s,16,16,7,2,17,4,"[{u'pos': 0, u'artist_name': u'The Smashing Pu..."
4,false,NaN,19156557,1430956800,Wedding,71,56,3,1,80,5,"[{u'pos': 0, u'artist_name': u'Cali Swag Distr..."
5,false,NaN,12674796,1509321600,2017,52,48,38,1,53,7,"[{u'pos': 0, u'artist_name': u'Fink', u'track_..."
6,false,NaN,9948921,1508976000,BOP,37,23,21,2,46,8,"[{u'pos': 0, u'artist_name': u'Catfish and the..."
7,false,NaN,4297488,1501804800,old country,20,18,10,1,21,9,"[{u'pos': 0, u'artist_name': u'Willie Nelson',..."
8,false,NaN,16403398,1509321600,abby,60,40,36,2,72,10,"[{u'pos': 0, u'artist_name': u'Post Malone', u..."
9,false,NaN,1981349,1481587200,relax,9,9,3,1,9,12,"[{u'pos': 0, u'artist_name': u'Yael Naim', u't..."


<h1> test_set </h1>

In [7]:
f = open('./test_R.json').read()
data = json.loads(f)
test_df = pd.DataFrame(data)

<h1> train_set_name & test_set_name </h1>

In [398]:
train_playlist_name = list(set(train_df['name']))
test_playlist_name = list(set(test_df['name']))

#train_playlist_name
#소문자로
train_texts = [[word for word in name.lower().split()]
         for name in train_playlist_name]

#test_playlist_name
#역시 소문자로
test_texts = [[word for word in name.lower().split()]
         for name in test_playlist_name]

In [242]:
train_texts


[[u'summer', u'country'],
 [u'this', u'is', u'what', u'you', u'came', u'for'],
 [u'winter'],
 [u'pregame'],
 [u'dance'],
 [u'relaxing', u'music'],
 [u'ouch'],
 [u'fuego'],
 [u'say', u'my', u'name'],
 [u'sleep'],
 [u'electronicas'],
 [u'lit', u'songs'],
 [u'the', u'mix'],
 [u'taylor', u'swift'],
 [u'mellow'],
 [u'classic', u'rock'],
 [u'music.'],
 [u'twenty', u'sixteen'],
 [u'frozen'],
 [u'guitar.'],
 [u'winter', u"'17"],
 [u'sweat.it.out.'],
 [u'dinner', u'party'],
 [u'beautiful', u'things'],
 [u'\U0001f608\U0001f608\U0001f608'],
 [u'tb'],
 [u'sound'],
 [u'feel', u'good'],
 [u'tennessee'],
 [u'bang', u'bang.'],
 [u'car', u'playlist'],
 [u'summer', u'playlist'],
 [u'tears.'],
 [u'throwback'],
 [u'stuff'],
 [u'trippin'],
 [u'thinking', u'of', u'you'],
 [u'rock\U0001f3b8'],
 [u'rock', u'en', u'espa\xf1ol'],
 [u"don't"],
 [u'\U0001f3b5\U0001f3b5'],
 [u'summer', u'drives'],
 [u'mixtape'],
 [u'vibes'],
 [u'road', u'trip', u'mix'],
 [u'wedding', u'songs'],
 [u'get', u'pumped'],
 [u'panda'],
 

In [56]:
test_texts
len(test_texts)

97

<h1> test using gensim_LDA </h1>

In [399]:
#create Dictionary
dictionary = corpora.Dictionary(train_texts)

#Dictionary encapsulates the mapping between normalized words and their integer ids.
corpus = [dictionary.doc2bow(text) for text in train_texts]

model = models.ldamodel.LdaModel(corpus =  corpus, id2word = dictionary, num_topics = 7,
                                random_state = 0, chunksize = 20000, passes = 10)

In [23]:
pd.DataFrame(corpus)

,0,1,2,3,4,5,6,7
0,"(0, 1)","(1, 1)",None,None,None,None,None,None
1,"(2, 1)","(3, 1)","(4, 1)","(5, 1)","(6, 1)","(7, 1)",None,None
2,"(8, 1)",None,None,None,None,None,None,None
3,"(9, 1)",None,None,None,None,None,None,None
4,"(10, 1)",None,None,None,None,None,None,None
5,"(11, 1)","(12, 1)",None,None,None,None,None,None
6,"(13, 1)",None,None,None,None,None,None,None
7,"(14, 1)",None,None,None,None,None,None,None
8,"(15, 1)","(16, 1)","(17, 1)",None,None,None,None,None
9,"(18, 1)",None,None,None,None,None,None,None


<h3> document 별 topic 다항분포 (topic = 7, train_set) </h3>

In [400]:
train_dic = {}
for idx in range(0, len(train_texts)):
    t = train_texts[idx]
    bow = dictionary.doc2bow(t)
    get_document_topics = tuple(model.get_document_topics(bow))
    name = ' '.join(t)
    train_dic[name] = dict(get_document_topics)

In [401]:
train_dic = pd.DataFrame(train_dic)
train_dic

,#boostyourrun,#mood,#relaxed,#workout,( ͡° ͜ʖ ͡°),((chris)),*** good stuff,//new,000,00s,...,🔥🔥🔥🔥,🖕🏻,😈😈😈,😍😍,😍😍😍,😭😭,😴😴,😻😻,🤘🏼,🤤🤤
0,0.071429,0.071429,0.071429,0.071429,0.028572,0.071429,0.035715,0.071429,0.071429,0.571426,...,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.571426
1,0.071429,0.071429,0.071429,0.071429,0.028572,0.071429,0.035722,0.571427,0.071429,0.071429,...,0.571427,0.071429,0.571427,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429
2,0.071429,0.071429,0.071429,0.071429,0.028572,0.571427,0.035715,0.071429,0.071429,0.071429,...,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.571427,0.071429,0.071429,0.071429
3,0.071429,0.071429,0.071429,0.571425,0.028572,0.071429,0.035714,0.071429,0.071429,0.071429,...,0.071429,0.071429,0.071429,0.571425,0.571425,0.071429,0.071429,0.571425,0.071429,0.071429
4,0.571426,0.571426,0.071429,0.071429,0.028572,0.071429,0.285131,0.071429,0.071429,0.071429,...,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429
5,0.071429,0.071429,0.571426,0.071429,0.028572,0.071429,0.536288,0.071429,0.071429,0.071429,...,0.071429,0.571426,0.071429,0.071429,0.071429,0.571426,0.071429,0.071429,0.071429,0.071429
6,0.071429,0.071429,0.071429,0.071429,0.828571,0.071429,0.035715,0.071429,0.571426,0.071429,...,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.071429,0.571426,0.071429


<h3> document 별 topic 다항분포 (topic = 7, test_set) </h3>

In [403]:
test_dic = {}
for idx in range(0, len(test_texts)):
    t = test_texts[idx]
    bow = dictionary.doc2bow(t)
    get_document_topics = tuple(model.get_document_topics(bow))
    name = ' '.join(t)
    test_dic[name] = dict(get_document_topics)

In [404]:
test_dic = pd.DataFrame(test_dic)
test_dic

,#chill,.jams.,2000s hits,<3,ale,ashley,awesome mix vol. 1,baby shower,bae,beautiful,...,trip,tropical,twenty one pilots,vibin,wake up,wedding cocktail hour,winter 2014,worship,xxxmas,💸💸
0,0.142857,0.142857,0.047619,0.071429,0.142857,0.142857,0.035715,0.047619,0.142857,0.071429,...,0.071429,0.142857,0.071429,0.571426,0.571233,0.785636,0.047620,0.571428,0.142857,0.142857
1,0.142857,0.142857,0.047619,0.571427,0.142857,0.142857,0.035715,0.047619,0.142857,0.071429,...,0.071429,0.142857,0.071429,0.071429,0.071429,0.035714,0.047620,0.071429,0.142857,0.142857
2,0.142857,0.142857,0.380893,0.071429,0.142857,0.142857,0.035715,0.047619,0.142857,0.571427,...,0.071429,0.142857,0.071429,0.071429,0.071429,0.035714,0.047620,0.071429,0.142857,0.142857
3,0.142857,0.142857,0.047619,0.071429,0.142857,0.142857,0.479398,0.047724,0.142857,0.071429,...,0.071859,0.142857,0.071429,0.071429,0.071429,0.035733,0.380643,0.071429,0.142857,0.142857
4,0.142857,0.142857,0.047619,0.071429,0.142857,0.142857,0.035714,0.047953,0.142857,0.071429,...,0.570996,0.142857,0.071429,0.071429,0.071429,0.035714,0.047619,0.071429,0.142857,0.142857
5,0.142857,0.142857,0.047619,0.071429,0.142857,0.142857,0.035714,0.713846,0.142857,0.071429,...,0.071429,0.142857,0.071429,0.071429,0.071429,0.035714,0.381259,0.071429,0.142857,0.142857
6,0.142857,0.142857,0.381011,0.071429,0.142857,0.142857,0.342030,0.047619,0.142857,0.071429,...,0.071429,0.142857,0.571426,0.071429,0.071624,0.035773,0.047619,0.071429,0.142857,0.142857


In [405]:
test_dic = test_dic.T
test_dic

,0,1,2,3,4,5,6
#chill,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857
.jams.,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857
2000s hits,0.047619,0.047619,0.380893,0.047619,0.047619,0.047619,0.381011
<3,0.071429,0.571427,0.071429,0.071429,0.071429,0.071429,0.071429
ale,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857
ashley,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857
awesome mix vol. 1,0.035715,0.035715,0.035715,0.479398,0.035714,0.035714,0.342030
baby shower,0.047619,0.047619,0.047619,0.047724,0.047953,0.713846,0.047619
bae,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857
beautiful,0.071429,0.071429,0.571427,0.071429,0.071429,0.071429,0.071429


In [77]:
test_dic[2][test_dic.index[0]]

0.1428571492433548

In [407]:
result = []
for trainName in train_dic.columns:
    idx = 0
    c = 0
    columns_ones = []
    for testName in test_dic.index:
        for idx in range(0, 7):
            c += pow(train_dic[trainName][idx] - test_dic[idx][testName], 2)
        c = 1/(1+sqrt(c))
        columns_ones.append(c)
    result.append(columns_ones)

<h3> train_playlist_name X test_playlist_name 유클리디안 거리 행렬 생성 </h3>

In [408]:
result = pd.DataFrame(result, columns = test_dic.index, index = train_dic.columns)

In [409]:
result

,#chill,.jams.,2000s hits,<3,ale,ashley,awesome mix vol. 1,baby shower,bae,beautiful,...,trip,tropical,twenty one pilots,vibin,wake up,wedding cocktail hour,winter 2014,worship,xxxmas,💸💸
#boostyourrun,0.683570,0.513465,0.502307,0.499713,0.542010,0.534859,0.491940,0.479165,0.545633,0.494423,...,0.575957,0.529393,0.496380,0.500454,0.499968,0.466925,0.508377,0.498958,0.542142,0.534837
#mood,0.683570,0.513465,0.502307,0.499713,0.542010,0.534859,0.491940,0.479165,0.545633,0.494423,...,0.575957,0.529393,0.496380,0.500454,0.499968,0.466925,0.508377,0.498958,0.542142,0.534837
#relaxed,0.683570,0.513465,0.502307,0.499713,0.542011,0.534859,0.491940,0.582053,0.528436,0.496496,...,0.494903,0.542849,0.494756,0.500658,0.499943,0.466927,0.563193,0.492341,0.543298,0.534647
#workout,0.683571,0.513465,0.502307,0.499713,0.542011,0.534859,0.558890,0.472268,0.546871,0.494275,...,0.496260,0.542612,0.494785,0.500654,0.499943,0.466929,0.563067,0.492356,0.543296,0.534648
( ͡° ͜ʖ ͡°),0.574496,0.485496,0.526047,0.457758,0.499212,0.494166,0.508482,0.442307,0.501146,0.459981,...,0.460255,0.498902,0.568512,0.454060,0.464348,0.436520,0.467616,0.463041,0.498557,0.494244
((chris)),0.683570,0.513465,0.553937,0.493434,0.543106,0.534679,0.491961,0.479127,0.545639,0.575151,...,0.494903,0.542849,0.494756,0.500658,0.499943,0.466927,0.508377,0.498958,0.542142,0.534837
*** good stuff,0.674716,0.512180,0.494356,0.494116,0.539848,0.532257,0.484097,0.569251,0.527603,0.490213,...,0.521166,0.535304,0.489326,0.494716,0.494095,0.460797,0.551606,0.487470,0.540989,0.532073
//new,0.683570,0.513465,0.502307,0.585228,0.527940,0.537194,0.491667,0.479158,0.545634,0.494423,...,0.494938,0.542843,0.494757,0.500658,0.499943,0.466927,0.508377,0.498958,0.542142,0.534837
000,0.683570,0.513465,0.553959,0.493432,0.543107,0.534679,0.534232,0.474744,0.546425,0.494328,...,0.494904,0.542849,0.575777,0.490871,0.501196,0.466813,0.508392,0.498956,0.542142,0.534837
00s,0.683570,0.513465,0.502307,0.499713,0.542010,0.534859,0.491940,0.479129,0.545639,0.494422,...,0.494904,0.542849,0.494756,0.587065,0.566187,0.559527,0.496575,0.586620,0.527723,0.537231


In [353]:
result['#chill']

#boostyourrun     0.679623
#mood             0.679623
#relaxed          0.679623
#workout          0.679623
( ͡° ͜ʖ ͡°)       0.570045
((chris))         0.679623
*** good stuff    0.585800
//new             0.679623
000               0.679623
00s               0.679623
14                0.679623
1967              0.679623
1980's            0.679623
1990's            0.679623
2000's            0.679623
2000s             0.679623
2000s r&b         0.614047
2012              0.679623
2013              0.679632
2015              0.679623
2016              0.679625
2017              0.679623
2020              0.679623
2k14              0.679623
2k17              0.679623
2pac              0.679623
4/20              0.679623
4th of july       0.585799
70/80             0.679623
80's              0.679623
                    ...   
wrap              0.679623
xmas              0.679623
xmas party        0.614079
xx                0.679623
yeah              0.679623
yeet              0.679623
y

In [354]:
result['#chill'].sort_values(ascending= False)

summer party                                       0.706375
summer '16                                         0.706373
my music                                           0.706364
car party                                          0.706360
country mix                                        0.706358
workout mix                                        0.706358
rap mix                                            0.706357
old people                                         0.706357
chill music                                        0.706355
run it                                             0.706353
90s hits                                           0.706353
my country                                         0.706353
my hits                                            0.706352
my favorites                                       0.706352
rock alternativo                                   0.706352
running 2                                          0.706352
garage rock                             

In [237]:
result_playlist_name = result['#chill'].sort_values(ascending= False).head(10).index
result_playlist_name

Index([u'songs that make me think of you', u'the good stuff', u'my favorites',
       u'classical favorites', u'christmas 2016', u'april 2016',
       u'smooth jams', u'rock songs', u'new jams', u'party mix'],
      dtype='object')

<h3> test_playlist_name -case "#chill" </h3>

In [181]:
train_df['name'][0].lower()

u'throwbacks'

In [238]:
train_playlist = []
for train_name in result_playlist_name:
    for idx in range(0,len(train_df['name'])):
        if(train_df['name'][idx].lower() == train_name):
            df = pd.DataFrame(train_df['tracks'][idx])
            train_playlist += list(df['track_uri'])

In [239]:
len(train_playlist)

658

In [246]:
test_playlist = []
test_track_uri = pd.DataFrame(test_df['tracks'][91])
test_playlist = test_track_uri['track_uri']

test_playlist = list(test_playlist)
test_playlist
len(test_playlist)

70

In [247]:
1.0 *len(set(train_playlist).intersection(set(test_playlist)))/len(set(test_playlist)) *100

5.797101449275362

In [280]:
result.columns

Index([u'#chill', u'.jams.', u'2000s hits', u'<3', u'ale', u'ashley',
       u'awesome mix vol. 1', u'baby shower', u'bae', u'beautiful',
       u'best songs of 2016', u'bored', u'brad paisley', u'bus playlist',
       u'chill out', u'christian music', u'christmas', u'christmas music',
       u'clubbin', u'corridos', u'country', u'december', u'disney',
       u'disneyland', u'edc', u'feelin' myself', u'feels', u'feelz', u'funky',
       u'game time', u'good music', u'good vibes', u'gospel', u'helene',
       u'hits', u'i workout!', u'in my feels', u'kendrick lamar', u'kyle',
       u'ladies', u'life', u'litty', u'love music', u'lulu', u'mat', u'maui',
       u'mellow.', u'mine', u'mix', u'modern rock', u'mom's playlist',
       u'mr. brightside', u'my high-intensity workout', u'office', u'old',
       u'party', u'party playlist', u'pop', u'pump', u'pump up songs', u'rage',
       u'rap', u'rap/pop', u'reggae', u'relax', u'rock', u'roma', u'sb2k17',
       u'september', u'shower', u'sle

In [291]:
len(set(test_df['name']))

97

<h3> test_allCase </h3>

In [410]:
count = 0
c = 0;
for name in result.columns:
    #test_playlist_name 과 거리가 가까운 train_playlist_name 상위 10개 도출
    result_playlist_name = result[name].sort_values(ascending = False).head(10).index
    print count
    count+=1
    
    #test_playlist 생성
    test_playlist = []
    for idx in range(0, len(test_df['name'])):
        if(test_df['name'][idx].lower() == name):
            test_track_uri = pd.DataFrame(test_df['tracks'][idx])
            test_playlist = list(test_track_uri['track_uri'])
    #print len(test_playlist)        
    if(len(test_playlist) == 0):
        continue
            
    #비교할 train_playlist 생성
    train_playlist = []
    for train_name in result_playlist_name:
        for idx in range(0,len(train_df['name'])):
            if(train_df['name'][idx].lower() == train_name):
                df = pd.DataFrame(train_df['tracks'][idx])
                train_playlist += list(df['track_uri'])
    #print len(train_playlist)      
    #r-precision 계산
    c += 1.0 *len(set(train_playlist).intersection(set(test_playlist)))/len(set(test_playlist)) *100
    
#avarage result
c = 1.0 * c/(count)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93


<h3> chunksize = 2, topic_num = 7 </h3>

In [315]:
c

6.367510848572158

<h1> if chunksize = 2000 ? </h1>

In [327]:
c

7.998734151705265

<h3> chunksize = 2000, topic_num = 9 </h3>

In [368]:
c

7.617062341229546

<h3> chunksize = 2000, topic_num = 11 </h3>

In [385]:
c

4.640980736170668

<h3> chunksize = 2000, topic_num = 17 </h3>

In [397]:
c

5.609661864773066

<h3> chuncksize = 20000, topic_num = 7 </h3>

In [411]:
c

8.029129288483382